In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
import spacetimeformer as stf
import pandas as pd

from data import preprocess

/home/vdesai/anaconda3/envs/bats/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import numpy as np
import pytorch_lightning as pl
import spacetimeformer as stf
import pandas as pd

from data import preprocess

In [3]:
class Args:
    def __init__(self, input_data_path):
        self.input_data_path = input_data_path
df, max_seq_len = preprocess.preprocess(Args("/home/vdesai/bats_data/training_files/data.csv"))

In [4]:
model_path = f"/home/vdesai/bats_data/models/not_removed_with_seed_1.ckpt"

In [6]:
model = stf.spacetimeformer_model.Spacetimeformer_Forecaster(max_seq_len = max_seq_len).load_from_checkpoint(model_path)

Forecaster
	L2: 0.001
	Linear Window: 0
	Linear Shared Weights: False
	RevIN: False
	Decomposition: False
GlobalSelfAttn: AttentionLayer(
  (inner_attention): PerformerAttention(
    (kernel_fn): ReLU()
  )
  (query_projection): Linear(in_features=200, out_features=200, bias=True)
  (key_projection): Linear(in_features=200, out_features=200, bias=True)
  (value_projection): Linear(in_features=200, out_features=200, bias=True)
  (out_projection): Linear(in_features=200, out_features=200, bias=True)
  (dropout_qkv): Dropout(p=0.0, inplace=False)
)
GlobalCrossAttn: AttentionLayer(
  (inner_attention): PerformerAttention(
    (kernel_fn): ReLU()
  )
  (query_projection): Linear(in_features=200, out_features=200, bias=True)
  (key_projection): Linear(in_features=200, out_features=200, bias=True)
  (value_projection): Linear(in_features=200, out_features=200, bias=True)
  (out_projection): Linear(in_features=200, out_features=200, bias=True)
  (dropout_qkv): Dropout(p=0.0, inplace=False)
)
L

In [7]:
bats_time_series = stf.data.CSVTimeSeries(
                        raw_df = df,
                        time_col_name = "TimeIndex",
                        time_features = ["hour", "minute", "second"],
                        ignore_cols = ["Filename", "NextDirUp", 'Path', 'Version', 'Filter', 'Preemphasis', 'MaxSegLnght'],
                        val_split = 0.1,
                        test_split = 0.1
                    )
# create a dataloader with the bats_time_series object
bats_dataset = stf.data.CSVTorchDset(
                    csv_time_series = bats_time_series,
                    split = "train",
                    context_points = max_seq_len - 2,
                    target_points = 2, 
                    time_resolution = 1
                )

In [8]:
scaler = bats_time_series.apply_scaling
inverse_scaler = bats_time_series.reverse_scaling

In [9]:
model.set_inv_scaler(inverse_scaler);
model.set_scaler(scaler);

In [15]:
    i = 0
    batch_size = 4
    batch = [bats_dataset[j] for j in range(i, min(i + batch_size, len(bats_dataset)))]

    # Unpacking and stacking data for batch processing
    x_c_batch, y_c_batch, x_t_batch, y_t_batch = zip(*batch)
    x_c_batch = torch.stack(x_c_batch)
    y_c_batch = torch.stack(y_c_batch)
    x_t_batch = torch.stack(x_t_batch)
    y_t_batch = torch.stack(y_t_batch)

    # Apply model's inverse scaler and predict in batches
    y_c_batch = torch.from_numpy(model._inv_scaler(y_c_batch.numpy())).float()
    y_t_batch = torch.from_numpy(model._inv_scaler(y_t_batch.numpy())).float()
    yhat_t_batch = model.predict(x_c_batch, y_c_batch, x_t_batch)

    # Process each item in the batch for DataFrame conversion
    for j in range(len(batch)):
        # Handle cases where the last batch might be smaller than batch_size
        if j >= yhat_t_batch.size(0):
            break

        # Preparing data for DataFrame
        predictions_data = torch.cat((x_c_batch[j], y_c_batch[j], x_t_batch[j], yhat_t_batch[j]), dim=1)
        predictions_df = pd.DataFrame(predictions_data.numpy(), columns=df_columns)
        predictions_df["FileIndex"] = i + j
        predictions = pd.concat([predictions, predictions_df], ignore_index=True)

        originals_data = torch.cat((x_c_batch[j], y_c_batch[j], x_t_batch[j], y_t_batch[j]), dim=1)
        originals_df = pd.DataFrame(originals_data.numpy(), columns=df_columns)
        originals_df["FileIndex"] = i + j
        originals = pd.concat([originals, originals_df], ignore_index=True)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 30 but got size 2 for tensor number 2 in the list.

In [14]:
bats_dataset[:1]

TypeError: cannot do positional indexing on Index with these indexers [[0]] of type list

In [21]:
    batch_index = 0
    # Process each batch
    batch = [bats_dataset[j] for j in range(batch_index, min(batch_index + batch_size, len(bats_dataset)))]

    # Stack tensors for batch processing
    x_c_batch = torch.stack([item[0] for item in batch])
    y_c_batch = torch.stack([torch.from_numpy(model._inv_scaler(item[1].numpy())).float() for item in batch])
    x_t_batch = torch.stack([item[2] for item in batch])
    y_t_batch = torch.stack([torch.from_numpy(model._inv_scaler(item[3].numpy())).float() for item in batch])

    # Model prediction for each batch
    yhat_t_batch = model.predict(x_c_batch, y_c_batch, x_t_batch)
    print(yhat_t_batch)
    print(yhat_t_batch.shape)
    print(x_c_batch.shape)
    print(x_t_batch.shape)
    for j in range(len(batch)):
        # Concatenating tensors for DataFrame creation
        predictions_data = torch.cat((x_c_batch[j], y_c_batch[j]), dim=1)
        predictions_data = torch.cat((predictions_data, torch.cat((x_t_batch[j], yhat_t_batch[j]), dim=1)), dim=0)

        originals_data = torch.cat((x_c_batch[j], y_c_batch[j]), dim=1)
        originals_data = torch.cat((originals_data, torch.cat((x_t_batch[j], y_t_batch[j]), dim=1)), dim=0)

        # Create DataFrame and append
        predictions_df = pd.DataFrame(predictions_data.numpy(), columns=df_columns)
        predictions_df["FileIndex"] = i
        predictions = pd.concat([predictions, predictions_df], ignore_index=True)

        originals_df = pd.DataFrame(originals_data.numpy(), columns=df_columns)
        originals_df["FileIndex"] = i
        originals = pd.concat([originals, originals_df], ignore_index=True)
        
        i += 1

tensor([[[ 1.7140e+07,  1.9569e+03,  1.0350e+02,  1.0163e+01,  2.9925e+00,
           3.1871e+01,  5.2405e+01,  4.1075e+01,  2.2181e+01,  3.5683e+01,
           5.6501e+01,  1.4881e+00,  4.7292e+01,  3.3411e+01,  5.1138e+01,
           3.2350e+01,  4.6652e+00,  2.1032e+00, -8.0119e+00, -2.8814e+00,
           1.2001e+01,  9.7250e-01,  5.1685e+00,  7.7711e+00,  3.2536e+00,
           4.7981e+00,  2.3182e+00,  8.0928e-01,  1.3651e+00,  1.2618e+00,
           2.0386e+00,  1.3144e+00,  3.5840e+01,  3.8885e-01,  3.8676e+01,
           2.7690e+01,  6.1999e+01,  3.4607e+01,  4.2694e+01,  3.6853e+01,
           5.4419e+01,  4.1260e+01,  4.1168e+01,  2.0631e+01,  1.2536e+01,
           1.0421e+01,  5.0778e+00,  1.5560e+00,  2.4897e+00,  2.6724e+00,
           1.1181e+00,  7.6737e+01,  1.1597e+04,  1.5191e+04,  1.1484e+04,
           6.9017e-01,  7.9876e-01,  8.3375e-01,  6.5975e-01,  5.0718e-03,
          -8.5961e-01,  3.8071e+00,  5.3853e-04, -7.1508e-01,  2.9458e+00,
           7.2212e+00,  4

NameError: name 'df_columns' is not defined

In [25]:
df = pd.DataFrame(x_c_batch[0])

In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
0,30.0,-1.000000,0.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
1,30.0,-0.508475,0.298421,-1.0,-0.754237,-0.508475,-0.262712,-0.016949
